In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import cv2
import os

data_path = 'data'
categories = os.listdir(data_path)
labels = [i for i in range(len(categories))]
label_dict = dict(zip(categories,labels))

print(label_dict)
print(labels)
print(categories)

{'without_mask': 0, 'with_mask': 1}
[0, 1]
['without_mask', 'with_mask']


In [3]:
img = cv2.imread('data/with_mask/with_mask_1.jpg')

In [4]:
img.shape

(350, 525, 3)

In [5]:
img_size = 100
data = []
target = []

for category in categories:
    folder_path = os.path.join(data_path, category)
    img_names = os.listdir(folder_path)
    
    for img_name in img_names:
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
    
        try:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, (img_size,img_size))
            data.append(resized)
            target.append(label_dict[category])
        except xception as e:
            print('Exception:', e)

In [6]:
import numpy as np

data = np.array(data)/255
data = np.reshape(data, (data.shape[0], img_size,img_size,1))

target = np.array(target)

In [7]:
from keras.utils import to_categorical
new_target = to_categorical(target)

In [8]:
new_target

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

## CNN

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dropout, Dense

In [10]:
model = Sequential([
    Conv2D(200,(3,3),input_shape=data.shape[1:], activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(100,(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, new_target,test_size=0.2)

In [12]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((6042, 100, 100, 1), (1511, 100, 100, 1), (6042, 2), (1511, 2))

In [13]:
history = model.fit(x_train, y_train, epochs=10)

Epoch 1/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 206s 1s/step - accuracy: 0.5930 - loss: 0.6907
Epoch 2/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 243s 1s/step - accuracy: 0.7474 - loss: 0.5177
Epoch 3/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 229s 1s/step - accuracy: 0.8266 - loss: 0.3786
Epoch 4/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 225s 1s/step - accuracy: 0.8618 - loss: 0.3156
Epoch 5/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 220s 1s/step - accuracy: 0.8985 - loss: 0.2453
Epoch 6/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 243s 1s/step - accuracy: 0.9121 - loss: 0.2136
Epoch 7/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 246s 1s/step - accuracy: 0.9127 - loss: 0.1987
Epoch 8/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 237s 1s/step - accuracy: 0.9377 - loss: 0.1565
Epoch 9/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 238s 1s/step - accuracy: 0.9418 - loss: 0.1358
Epoch 10/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 213s 1s/step - accuracy: 0.9517 - loss: 0.1199


In [14]:
model.evaluate(x_test, y_test)

48/48 ━━━━━━━━━━━━━━━━━━━━ 18s 366ms/step - accuracy: 0.9003 - loss: 0.2642


[0.3065589964389801, 0.8848444819450378]

In [25]:
model.save('cnn_model.keras')